Configuração Inicial e Contexto

In [1]:
!pip install pyspark #Instala o PySpark no ambiente Colab, permitindo que o Python interaja com o Apache Spark

In [2]:
import pyspark #Importa a biblioteca PySpark para acesso às suas funcionalidades
from pyspark import SparkContext, SparkConf #Importa as classes principais: SparkContext (ponto de entrada) e SparkConf (para configurar o contexto)

In [3]:

conf = SparkConf().setAppName("leitura de arquivo texto") #Cria uma configuração (SparkConf) e define um nome amigável para a aplicação Spark (neste caso, "leitura de arquivo texto")
sc = SparkContext(conf=conf).getOrCreate() #Cria ou obtém o SparkContext. Ele usa a configuração definida (conf) e é o objeto que conecta o código ao cluster Spark

Leitura de Dados

In [5]:
rdd = sc.textFile("sample_data/README.md") #Cria um RDD lendo o conteúdo de um arquivo de texto (README.md) linha por linha. Cada linha do arquivo se torna um elemento no RDD

Ações e Transformações Básicas

In [6]:
rdd.count() #Retorna o número total de linhas (elementos) no RDD.

19

In [7]:
rdd.take(10) #Retorna uma lista com as primeiras 10 linhas do arquivo de texto.

['This directory includes a few sample datasets to get you started.',
 '',
 '*   `california_housing_data*.csv` is California housing data from the 1990 US',
 '    Census; more information is available at:',
 '    https://docs.google.com/document/d/e/2PACX-1vRhYtsvc5eOR2FWNCwaBiKL6suIOrxJig8LcSBbmCbyYsayia_DvPOOBlXZ4CAlQ5nlDD8kTaIDRwrN/pub',
 '',
 '*   `mnist_*.csv` is a small sample of the',
 '    [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), which is',
 '    described at: http://yann.lecun.com/exdb/mnist/',
 '']

In [10]:
palavra = rdd.flatMap(lambda x:x.split(' ')) #(flatMap): Transforma o RDD de linhas (rdd) em um RDD de palavras individuais.
                                             #O split(' ') divide cada linha por espaços, e o flatMap achata o resultado, garantindo que a saída seja um único RDD de palavras, em vez de um RDD de listas de palavras.

In [11]:
palavraMinuscula = palavra.map(lambda x: x.lower()) #(map): Aplica a função lower() (converter para minúsculas) a cada palavra no RDD, criando um novo RDD de palavras em minúsculas
print('Map: ', palavraMinuscula.take(5)) #Imprime as 5 primeiras palavras do RDD em minúsculas.

palavraMinusculaFlatMap = palavra.flatMap(lambda x: x.upper()) #(flatMap): Aplica a função upper() (converter para maiúsculas) a cada palavra.
                                                               #Importante: Como flatMap espera uma sequência de retorno, ele quebra cada palavra em caracteres individuais em maiúsculas

print('flatMap: ', palavraMinusculaFlatMap.take(5)) #Imprime os primeiros 5 caracteres resultantes do flatMap em maiúsculas.

Map:  ['this', 'directory', 'includes', 'a', 'few']
flatMap:  ['T', 'H', 'I', 'S', 'D']


In [12]:
palavraComecaT = palavraMinuscula.filter(lambda x: x.startswith('t')) #(filter): Cria um novo RDD contendo apenas as palavras que começam com a letra 't'.
print(palavraComecaT.take(5)) # Exibe as 5 primeiras palavras que começam com 't'.

palavraMin2 = palavraMinuscula.filter(lambda x: len(x) > 2) # (filter): Cria um novo RDD removendo palavras com 1 ou 2 caracteres (geralmente preposições, artigos ou strings vazias)
print(palavraMin2.take(5)) #Exibe as 5 primeiras palavras com mais de 2 caracteres.

['this', 'to', 'the', 'the', 'the']
['this', 'directory', 'includes', 'few', 'sample']


Contagem de Palavras (Word Count)

In [13]:
palavraChaveValor = palavraMin2.map(lambda x: (x,1)) #(map):Converte o RDD de palavras em um RDD de pares chave-valor (Tuplas). Cada palavra se torna a chave e o valor 1 é anexado, preparando para a contagem
palavraChaveValor.take(5) #Exibe os 5 primeiros pares chave-valor.

[('this', 1), ('directory', 1), ('includes', 1), ('few', 1), ('sample', 1)]

In [14]:
palavraContar = palavraChaveValor.reduceByKey(lambda x, y: x+y) # (reduceByKey): Agrupa todos os valores pela chave (palavra) e aplica uma função de agregação (neste caso, soma os valores 1), resultando na contagem final de cada palavra
palavraContarOrd = palavraContar.sortByKey(ascending=-1) #(sortByKey): Ordena o resultado da contagem de palavras com base na chave (a própria palavra)
palavraContar.take(20) #Exibe as 20 primeiras palavras e suas contagens, na ordem em que o Spark as processou (não ordenadas).

[('this', 1),
 ('sample', 2),
 ('datasets', 1),
 ('you', 1),
 ('started.', 1),
 ('`california_housing_data*.csv`', 1),
 ('from', 1),
 ('1990', 1),
 ('more', 1),
 ('information', 1),
 ('available', 1),
 ('small', 1),
 ('database](https://en.wikipedia.org/wiki/mnist_database),', 1),
 ('described', 2),
 ("[anscombe's", 1),
 ('statistical', 1),
 ("analysis'.", 1),
 ('american', 1),
 ('jstor', 1),
 ('2682899.', 1)]

Salvamento e Leitura de Resultados

In [15]:
palavraContar.saveAsTextFile("contar_palavras_out") #O Spark salvará o resultado em vários arquivos de saída dentro deste diretório, um por partição.

In [16]:
!ls contar_palavras_out/ #Comando Shell: Lista o conteúdo do diretório de saída para confirmar que os arquivos foram criados

part-00000  part-00001	_SUCCESS


In [20]:
rddContarPalavras = sc.textFile("contar_palavras_out") #Cria um novo RDD lendo os arquivos de texto que acabaram de ser salvos no diretório de saída.